# 1. Import dependencies

In [1]:
import copy
import cv2 as cv
import csv
import mediapipe as mp
import numpy as np
import itertools

from collections import deque

# 2. Initialization

## a. Initialize variables and models

In [2]:
webcam = 0
cap_width = 960
cap_height = 540

p_time = 0
c_time = 0

# Camera preparation ########################################################
cap = cv.VideoCapture(webcam)
cap.set(cv.CAP_PROP_FRAME_WIDTH, cap_width)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, cap_height)

# Mediapipe hand model load #################################################
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5,
)

# Coordinate history ########################################################
history_length = 16

## b. Initialize functions

In [3]:
# Calculate landmark list ###################################################
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [4]:
# Pre-processing calculated landmarks ########################################
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

## c. Initialize dataset template function

In [5]:
num_coords = 21
alphabet = "B"

In [6]:
def init_csv(feature):
    landmarks = ['class']
    for val in range(1, num_coords+1):
        landmarks += ['x{}'.format(val), 'y{}'.format(val)]

    with open('dataset/keypoints_{}.csv'.format(feature), mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)

In [7]:
init_csv(alphabet)

## d. Initialize dataset logging function

In [8]:
def logging_csv(feature, landmark_list):

    csv_path = 'dataset/keypoints_{}.csv'.format(feature)
    with open(csv_path, 'a', newline="") as f:
        writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([feature, *landmark_list])

    return

# 3. Capture, re-normalize & write landmark keypoints into dataset

In [9]:
# During capturing process ##################################################
while True:
    # FPS measurement #######################################################
#     c_time = time.time()
#     fps = 1 / (c_time - p_time)
#     p_time = c_time

    # Exit argument #########################################################
    if cv.waitKey(5) & 0xFF == 27:  # ESC key.
        break

    # If frame in capture is available: #####################################
    available, image = cap.read()
    if not available:
        break
    image = cv.flip(image, 1)  # Mirror display
    debug_image = copy.deepcopy(image)

    # Apply Mediapipe into detection ########################################
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

    # Optimize detection process ############################################
    image.flags.writeable = False
    results = hands.process(image)

    # If the hand is detected: ##############################################
    image.flags.writeable = True

    if results.multi_hand_landmarks is not None:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks,
                                              results.multi_handedness):
            # Bounding box calculation
#             bounding_box = calc_bounding_box(debug_image, hand_landmarks)

            # Landmark calculation by actual image size
            landmark_list = calc_landmark_list(debug_image, hand_landmarks)

            # Conversion into relative coordinates / normalized coordinates (wrist point)
            pre_processed_landmark_list = pre_process_landmark(landmark_list)
            print(pre_processed_landmark_list)

            # Write to csv dataset
            logging_csv(alphabet, pre_processed_landmark_list)

            # Draw into image output
            mp_drawing.draw_landmarks(
                debug_image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
    # Output ################################################################
    cv.imshow('Hand Gesture Recognition', debug_image)
    
cap.release()
cv2.destroyAllWindows()

[0.0, 0.0, -0.358695652173913, -0.10869565217391304, -0.6413043478260869, -0.3695652173913043, -0.7282608695652174, -0.6847826086956522, -0.75, -0.9347826086956522, -0.5869565217391305, -0.7065217391304348, -0.6304347826086957, -1.0, -0.5760869565217391, -0.782608695652174, -0.5434782608695652, -0.5869565217391305, -0.3695652173913043, -0.75, -0.3804347826086957, -0.9565217391304348, -0.33695652173913043, -0.6847826086956522, -0.32608695652173914, -0.5217391304347826, -0.16304347826086957, -0.7717391304347826, -0.17391304347826086, -0.9347826086956522, -0.14130434782608695, -0.6847826086956522, -0.14130434782608695, -0.5108695652173914, 0.043478260869565216, -0.7608695652173914, 0.021739130434782608, -0.9239130434782609, 0.021739130434782608, -0.75, 0.021739130434782608, -0.6086956521739131]
[0.0, 0.0, -0.33695652173913043, -0.10869565217391304, -0.5978260869565217, -0.391304347826087, -0.7065217391304348, -0.6847826086956522, -0.7717391304347826, -0.9130434782608695, -0.54347826086956

[0.0, 0.0, -0.33707865168539325, -0.07865168539325842, -0.5955056179775281, -0.3595505617977528, -0.7078651685393258, -0.651685393258427, -0.7865168539325843, -0.8876404494382022, -0.550561797752809, -0.7078651685393258, -0.6292134831460674, -1.0, -0.5955056179775281, -0.8089887640449438, -0.550561797752809, -0.651685393258427, -0.33707865168539325, -0.7415730337078652, -0.39325842696629215, -0.9550561797752809, -0.34831460674157305, -0.6629213483146067, -0.3146067415730337, -0.5393258426966292, -0.12359550561797752, -0.7415730337078652, -0.1797752808988764, -0.8764044943820225, -0.15730337078651685, -0.6067415730337079, -0.12359550561797752, -0.5056179775280899, 0.10112359550561797, -0.7191011235955056, 0.02247191011235955, -0.8539325842696629, 0.011235955056179775, -0.651685393258427, 0.033707865168539325, -0.5730337078651685]
[0.0, 0.0, -0.34444444444444444, -0.1, -0.6111111111111112, -0.3888888888888889, -0.7111111111111111, -0.6777777777777778, -0.7777777777777778, -0.9, -0.533333

[0.0, 0.0, -0.3333333333333333, -0.10752688172043011, -0.5806451612903226, -0.3763440860215054, -0.6881720430107527, -0.6881720430107527, -0.7419354838709677, -0.9354838709677419, -0.5591397849462365, -0.7311827956989247, -0.6236559139784946, -1.0, -0.5698924731182796, -0.7849462365591398, -0.5161290322580645, -0.6129032258064516, -0.3655913978494624, -0.7849462365591398, -0.40860215053763443, -0.989247311827957, -0.34408602150537637, -0.7096774193548387, -0.3010752688172043, -0.5591397849462365, -0.16129032258064516, -0.7956989247311828, -0.21505376344086022, -0.956989247311828, -0.17204301075268819, -0.6881720430107527, -0.13978494623655913, -0.5376344086021505, 0.03225806451612903, -0.7741935483870968, -0.03225806451612903, -0.8924731182795699, -0.03225806451612903, -0.6989247311827957, 0.0, -0.5698924731182796]
[0.0, 0.0, -0.358695652173913, -0.13043478260869565, -0.6086956521739131, -0.41304347826086957, -0.7065217391304348, -0.7065217391304348, -0.782608695652174, -0.956521739130

NameError: name 'cv2' is not defined